In [1]:
import os
import datetime
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [2]:
from typing import List, Optional

In [3]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(openai_api_key=openai_api_key, model_name="gpt-4") #model_name="gpt-3.5-turbo")

In [4]:
task = "I want a website where I can track my weight"

In [5]:
# 1. Make detailed requirements list & ask clarifying questions
# 2. Decide on tech stack & architecture
# 3. Create files & code skeleton (ie signatures of main function + short describtion)
# 4. Create test files & define test cases
# 5. For each feature:
# 6. -- Implement code & test at each step if it works
# 7. -- Write unit test & make sure they pass
# 8. Write integration test
# 9. For each integration test: make sure they pass
# 10. Deploy
# ---
# 11. Ask for user feedback ("Is this exactly how you want it? Can I make it any better for you?")

In [6]:
class RunManager():
    file = "runs.txt"

    def __init__(self):
        if os.path.exists(self.file):
            self.runs = pd.read_csv(self.file)
        else:
            self.runs = pd.DataFrame(columns=["run_no", "time"])
            self.save()

    @property
    def run_no(self):
        return 0 if self.runs.empty else self.runs.run_no.max()

    def start_run(self):
        now = datetime.datetime.now().strftime("%Y-%m-%dT%H-%M")
        self.runs.loc[len(self.runs)] = {"run_no": self.run_no+1, "time": now}
        self.save()

    def save(self):
        self.runs.to_csv(self.file, index=False)

In [7]:
run_manager = RunManager()
run_manager.start_run()

In [8]:
# prompt files for easier access
UNDERSTAND = "1_understand_requirement"
ARCHITECTURE = "2_choose_architecture"
SKELETON = "3_create_skeleton"

# short names for logging
phase_names_for_logging = {
    UNDERSTAND: "1_understand",
    ARCHITECTURE: "2_architecture",
    SKELETON: "3_skeleton",
}

In [9]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator

In [10]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import HumanMessage, BaseMessage

def load_prompt(prompt_file: str) -> ChatPromptTemplate:
    path_to_prompt = f"prompts/{prompt_file}.txt"
    with open(path_to_prompt, "r") as file:
        file_contents = file.read()
    # todo: use system msg 
    human_msg = HumanMessagePromptTemplate.from_template(file_contents)
    return ChatPromptTemplate.from_messages([human_msg])

def get_prompt(prompt_file: str, **prompt_vars) -> List[BaseMessage]:
    prompt_template = load_prompt(prompt_file)
    return prompt_template.format_prompt(**prompt_vars).to_messages()

In [11]:
stage_for_logging = {
    "ideation": "__a_initial",
    "critique": "__b_critique",
    "resolution": ""
}
def save_llm_output(content, phase, stage):
    filename = f"cache/run_{run_manager.run_no}__{phase_names_for_logging[phase]}{stage_for_logging[stage]}.txt"
    with open(filename, "w") as file:
            file.write(content)

def get_thoughtful_reponse(prompt_file: str, verbose=True, save=True, save_intermediate=True, format_instructions=None, **prompt_vars) -> str:
    # Step 1: Initial response
    initial_response = llm(get_prompt(prompt_file, **prompt_vars)).content
    if verbose: print(f"> Initial response:\n{initial_response}\n")    
    if save_intermediate: save_llm_output(initial_response, phase=prompt_file, stage="ideation")
        
    # Step 2: Self-critique
    critique = llm(get_prompt(prompt_file+"__reflect", initial_response=initial_response, **prompt_vars)).content
    if verbose: print(f"> Self-critique:\n{critique}\n")
    if save_intermediate: save_llm_output(critique, phase=prompt_file, stage="critique")
    
    # Step 3: Thoughtful response
    if format_instructions: prompt_vars["format_instructions"] = format_instructions  # only use format instructions in last step
    thoughtful_response = llm(get_prompt(prompt_file+"__resolve", initial_response=initial_response, critique=critique, **prompt_vars)).content
    if verbose: print(f"> Thoughtful response:\n{thoughtful_response}\n")
    if save: save_llm_output(thoughtful_response, phase=prompt_file, stage="resolution")
    
    return thoughtful_response

### 1. Make detailed requirements list & ask clarifying questions

In [12]:
# TODO: what if we have questions?

In [13]:
project_description = get_thoughtful_reponse(UNDERSTAND, user_goal=task, save_to="1_understand")

> Initial response:
The user's goal is to have a website to track their weight over time.

Therefore, the user needs:
- A registration and login system, so the data is personalized and secure
- A profile page where the user can input and edit their personal information (name, age, gender, height)
- A dashboard where the user can input their daily weight and view a chart of their weight progress over time
- The ability to set weight goals, such as target weight or rate of weight loss/gain
- Notifications or reminders to input daily weight, if the user chooses to enable them
- A responsive design that works on both desktop and mobile devices

In this, I have made the following assumptions:
- The user has access to a reliable internet connection
- The user is comfortable with providing personal information for the purpose of tracking their weight
- The user's primary motivation for using the website is to track their weight, not to receive additional health advice or recommendations

I ha

### 2. Decide on tech stack & architecture

In [14]:
# todo: include description of replit auth + storage, as that simplifies architecture

In [15]:
architecture = get_thoughtful_reponse(ARCHITECTURE, project_description=project_description, save_to="2_architecture")

> Initial response:
(i) Architecture

For this project, a simple yet effective architecture would be the Model-View-Controller (MVC) architecture. The MVC architecture would allow us to separate our concerns easily, manage the code efficiently, and scale the application if needed in the future.

1. Model: Represents the data and the business logic of the application. In this case, it will include the user's personal information, weight data, and weight goals.
2. View: Represents the user interface and the presentation of the data. It will include the registration/login, profile, and dashboard pages.
3. Controller: Handles user input and updates the model and view accordingly. It will manage user authentication, data input, data retrieval, and data export.

(ii) Tech Stack

Backend:
1. Language: Python, due to its versatility and ease of use.
2. Framework: Django, a robust and easy-to-use Python web framework that follows the MVC architecture.
3. Database: PostgreSQL, a powerful and sca

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIError: The server had an error while processing your request. Sorry about that! {
  "error": {
    "message": "The server had an error while processing your request. Sorry about that!",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'The server had an error while processing your request. Sorry about that!', 'type': 'server_error', 'param': None, 'code': None}} {'Date': 'Thu, 08 Jun 2023 17:19:45 GMT', 'Content-Type': 'application/json', 'Content-Length': '176', 'Connection': 'keep-alive', 'access-control-allow-origin': '*', 'openai-model': 'gpt-4-0314', 'openai-organization': 'user-y6xsbu6zlupjxqpx8sdyagib', 'openai-processing-ms': '565', 'openai-version': '2020-10-01', 'strict-transport-security': 'max-age=15724800; includeSubDomains', 'x-ratelimit-limit-requests': '200', 'x-ratelimit-limit-tokens': '400

> Thoughtful response:
An appropriate architecture and tech stack for this project is as follows:

(i) Architecture: Model-View-Controller (MVC) architecture, allowing for efficient separation of concerns, code management, and scalability.

(ii) Tech Stack:

Backend:
1. Language: Python, for its versatility and ease of use.
2. Framework: Django, a robust Python web framework that follows the MVC architecture and includes built-in authentication support.
3. Database: PostgreSQL, a powerful and scalable open-source relational database.

Frontend:
1. HTML, CSS, and JavaScript for building the user interface.
2. Bootstrap, for achieving a responsive design that works on both desktop and mobile devices.
3. Chart.js for creating interactive weight progress charts.

Authentication:
1. Django's built-in authentication system.

APIs and Libraries:
1. Django REST framework for creating RESTful APIs if needed.
2. Pandas (Python library) for handling data export to CSV or Excel files.

Notificatio

### 3. Create files & code skeleton (ie signatures of main function + short describtion)

In [16]:
class Function(BaseModel):
    name: str = Field(description="name of function")
    signature: str = Field(description="signature of function")
class CodeFile(BaseModel):
    name: str = Field(description="full path to this file")
    description: str = Field(description="what does the code in this file do?")
    functions: Optional[List[Function]] = Field(description="functions in this file")    
class CodeBase(BaseModel):
    code_files: List[CodeFile] = Field(description="a code file")
code_file_parser = PydanticOutputParser(pydantic_object=CodeBase)

In [17]:
skeleton = get_thoughtful_reponse(
    SKELETON,
    project_description=project_description,
    architecture=architecture,
    format_instructions=code_file_parser.get_format_instructions(),
    save_to="3_skeleton"
)

> Initial response:
1. `models.py` - Contains the database models for the user profile, weight data, and weight goals.

   Functions:
   - `class UserProfile(models.Model):` - Represents the user profile with personal information.
   - `class WeightData(models.Model):` - Represents the daily weight data input by the user.
   - `class WeightGoal(models.Model):` - Represents the user's weight goals (target weight or rate of weight loss/gain).

2. `views.py` - Contains the view functions that handle HTTP requests and responses.

   Functions:
   - `def register(request):` - Handles user registration.
   - `def login(request):` - Handles user login.
   - `def logout(request):` - Handles user logout.
   - `def profile(request):` - Renders the user profile page.
   - `def edit_profile(request):` - Handles editing the user profile information.
   - `def dashboard(request):` - Renders the dashboard page with weight progress chart and goals.
   - `def add_weight_data(request):` - Handles adding

In [18]:
parsed_skeleton = code_file_parser.parse(skeleton)

In [24]:
def create_project(codebase: CodeBase) -> None:
    home_dir = f"output/run_{run_manager.run_no}/"
    os.makedirs(home_dir, exist_ok=True)
    
    for code_file in parsed_skeleton.code_files:
        
        comment = "# " if code_file.name.endswith(".py") else "// "
        
        with open(home_dir+code_file.name, "w") as f:
            f.write(comment+code_file.description+'\n\n')
            for func in code_file.functions:
                f.write(comment+func.signature+'\n\n')

In [25]:
create_project(parsed_skeleton)

### 4. Create test files & define test cases

### 5./6./7. For each feature: Implement code & test at each step if it works // Write unit test & make sure they pass

### 8. Write integration test

### 9. For each integration test: make sure they pass

### 10. Deploy

### Footer